## Finetuning a `GPT-3` model with training data generated by `ChatGPT` 

Finetuning a GPT-like model is not what usually comes to mind. It is not a practice of feeding it new data, or retraining it on your own corpus. It's rather guiding it towards the desired output format, length, content, structure. 

I will be creating the training data out of ChatGPT's responses, then I will filter for acceptable / good quality prompt-completion pairs, then finally I'll finetune a GPT-3 model for that certain task.

The desired model will be one that suggests team building exercises given a (1) number of people, (2) their profession, (3) desired activities.

In [22]:
import credentials
import time
import json
import random
import os
os.environ["OPENAI_API_KEY"] = credentials.openai_api

from langchain.chat_models import ChatOpenAI

from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

import openai

### 1. Creating the training data

Using ChatGPT via the LangChain API

In [39]:
chat = ChatOpenAI(temperature = 0.5)

system_message="""You are a consultant at a team building and team bonding event planning company. Your job is to come up with custom activities
and funtivities tailored to your client's needs. You take into consideration the number of people, their professions and desired activities.
You make long, detailed plans, covering at least 6 hours of activities. You try to include mental exercises as well as sporty, physical activities. 

You present your plan as a structured list of activities. Avoid banter and small-talk, just present your plan. 
Include some breaks for food and refreshments. Be very specific with every exercise, if necessary, present a step-by-step guide.

Here is the information you need to use:

"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)

human_template="""Number of people: {num_people}
Profession: {profession}
Preferred activities: {preferred_activities}"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [40]:
result = chain.run(num_people = '5-7', profession = 'Marketing', preferred_activities = 'Board games, basketball')
print(result)

Team Building and Team Bonding Event Plan for Marketing Professionals

Duration: 6 hours

Activities:

1. Icebreaker activity - "Two Truths and a Lie" - 30 minutes
   - Each participant introduces themselves and shares two true statements and one false statement about themselves. 
   - The group has to guess which statement is false.

2. Board game tournament - 2 hours
   - Divide the group into teams of two.
   - Provide a selection of board games, such as Monopoly, Scrabble, and Settlers of Catan.
   - Each team plays each other in a round-robin style tournament.
   - The team with the most wins at the end of the tournament is declared the winner.

3. Lunch break - 1 hour
   - Provide a catered lunch for the group.

4. Mental exercise - "Escape Room" - 1.5 hours
   - The group is divided into teams of 3-4 people.
   - Each team is given a set of clues to solve a series of puzzles and riddles to escape the room.
   - The team that escapes the room in the shortest amount of time is dec

Now we need to create 200-300 examples, which will then be checked by a human for their length and content

In [41]:
num_people = ['5', '7', '10-15', '20-30', '10', '20', 'around 10', 'less than 15', 'more than 20', 'approx. 15', '5-10']
professions = ['marketing', 'data scientists', 'financial advisors', 'consultants', 'developers', 'software developers',
               'grocery store employees', 'management', 'nurses', 'doctors', 'drivers', 'kitchen staff', 'factory workers',
               'football coaches', 'kindergarden teachers', 'high school teachers', 'salesmen']
desired_activities = ['basketball', 'football', 'board games', 'sauna', 'jacuzzi', 'brainteasers', 'table tennis', 
                      'cooking', 'hiking', 'escape room', 'food tasting', 'quiz', 'karaoke']

In [44]:
for i in range(300):

    num_p = random.choice(num_people)
    prof = random.choice(professions)
    act = random.choice(desired_activities)

    prompt = human_message_prompt.format_messages(num_people = num_p, profession = prof, preferred_activities = act)[0].content
    result = chain.run(num_people = num_p, profession = prof, preferred_activities = act)

    with open('../docs/gpt_generations/prompts/' + str(i) + '.txt', 'w') as f:
        f.write(prompt)

    with open('../docs/gpt_generations/outputs/' + str(i) + '.txt', 'w') as f:
        f.write(result)

    time.sleep(16)


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


### 2. Create `jsonl` file that will be passed to OpenAI thru the OpenAI finetuning API

Out of the 300 AI generated prompt-output pairs after human checking I deleted 12. Now I'm reading in the remaining as pairs to create the training file

In [55]:
final_files = os.listdir('../docs/gpt_generations/outputs')
final_files_indices = [i.split('.txt')[0] for i in final_files]

training_data = []

for i in final_files_indices:

    with open('../docs/gpt_generations/prompts/' + str(i) + '.txt') as f:
        prompt = f.read()

    with open('../docs/gpt_generations/outputs/' + str(i) + '.txt') as f:
        completion = f.read()

    training_data.append({'prompt' : prompt, 'completion' : completion})

In [57]:
training_data[5]

{'prompt': 'Number of people: 10\nProfession: software developers\nPreferred activities: karaoke',
 'completion': 'Here is a plan for a 6-hour team building event for 10 software developers who enjoy karaoke:\n\n10:00 AM - 10:30 AM: Arrival and Welcome\n- Guests arrive, get name tags and are welcomed by the event coordinator.\n- Refreshments are available.\n\n10:30 AM - 11:30 AM: Icebreaker Games\n- A few icebreaker games will be played to help everyone get to know each other better.\n- Games will include "Two Truths and a Lie" and "Human Bingo."\n\n11:30 AM - 12:30 PM: Karaoke Workshop\n- A professional karaoke instructor will lead a workshop on how to improve singing skills.\n- Participants will learn about breathing techniques, voice projection and how to choose the right song.\n\n12:30 PM - 1:30 PM: Lunch Break\n- A lunch buffet will be served.\n- Participants can socialize and network.\n\n1:30 PM - 2:30 PM: Escape Room Challenge\n- Participants will be divided into two groups and 

In [58]:
file_name = "../docs/gpt_generations/training_data.jsonl"

with open(file_name, 'w') as outfile:
    for entry in training_data:
        json.dump(entry, outfile)
        outfile.write('\n')

### 3. Validate training data

In [60]:
!openai tools fine_tunes.prepare_data -f ../docs/gpt_generations/training_data.jsonl

### 4. Start fine-tuning job

In [ ]:
!openai api fine_tunes.create \
--training_file ../docs/gpt_generations/training_data.jsonl \
--model davinci 
--suffix "TeamBuildingConsultant" \
--n_epochs 3

### 5. Check results

In [ ]:
!openai api fine_tunes.results -i <YOUR_FINE_TUNE_JOB_ID>

### 6. Use fine-tuned model with the `OpenAI` API

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="adasd",
  prompt="asdasd",
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["END"]
)